[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/elcursodeia-code/blob/master/dl/05_entrenamiento.ipynb)

# Entrenando Redes Neuronales

## Conjuntos de datos